In [8]:
from postgres_connect import *
path = '/Users/apple/Documents/tunnel-ssh.cer'

In [9]:
conn = get_conn('Yes',path)
query_path = 'rpr.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
df_report = pd.read_sql(custom_query,conn)
conn.close()

Tunnel Started
Connection Made


In [3]:
df_report

,shipping_city,hub,rider_id,rider_name,load,delivery_per,fasr,prepaid_fasr,cod_fasr,failed_collection_pendency,cod_collection_pendency,mtd_fake,pip_result
0,Bangalore,BLR FRH ECTY,Vishnu_Prasad__10,Vishnu Prasad,60,78.0,77.0,88.0,20.0,0,6505.0,NaN,None
1,Bangalore,BLR FRH ECTY,Santhu_7022177696_010,Santhu,5,80.0,80.0,80.0,NaN,0,0.0,NaN,None
2,Bangalore,BLR FRH ECTY,Raghu_K_R_10,Raghu K R,59,80.0,80.0,91.0,59.0,0,0.0,NaN,None
3,Bangalore,BLR FRH ECTY,Shekhar_vishnu_10,Shekhar vishnu,77,83.0,83.0,94.0,61.0,0,0.0,NaN,None
4,Bangalore,BLR FRH ECTY,Prakash_9686465303_010,Prakash M,14,86.0,86.0,100.0,50.0,1,1800.0,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,NCR,Noida-Franchise,Vikas_singh_20,Vikas singh,38,84.0,89.0,93.0,75.0,0,0.0,NaN,None
688,NCR,Noida-Franchise,Sumit_kashyap_20,Sumit kashyap,36,89.0,90.0,96.0,60.0,0,1715.0,NaN,None
689,NCR,Noida-Franchise,Harsh_kumar_20,Harsh kumar,48,69.0,94.0,100.0,82.0,3,250.0,NaN,None
690,NCR,Noida-Franchise,Rahul_9205549489_020,Rahul Solanki ks,36,92.0,94.0,100.0,71.0,3,0.0,NaN,None


In [10]:
# 1. Rename columns
df_report.rename(columns={
    'shipping_city': 'City',
    'hub': 'Hub',
    'rider_id': 'Rider ID',
    'rider_name': 'Name',
    'load': 'Load',
    'delivery_per': 'Del%',
    'fasr': 'FASR',
    'prepaid_fasr': 'Prepaid FASR',
    'cod_fasr': 'COD FASR',
    'failed_collection_pendency': 'Failed Pending',
    'cod_collection_pendency': 'COD Pending',
    'mtd_fake': 'MTD Fake',
    'pip_result': 'PIP Status'
}, inplace=True)

# 2. Remove NaN and None, make them blank
df_report=df_report.apply(lambda x: x.fillna(''))


In [11]:
city_dict1 = {
    'Bangalore':['mayank@blitznow.in','anil@blitznow.in','ma.saifullah@blitznow.in','syed.afrid@blitznow.in','dilip.kumar@blitznow.in','biswojit.jena@blitznow.in'],
    'Delhi':['mayank@blitznow.in','sanjay@blitznow.in','ajay.verma@blitznow.in','rahul.lochav@blitznow.in'],
    'Mumbai':['mayank@blitznow.in','anil@blitznow.in','david.kumar@blitznow.in','sumit.kamble@blitznow.in'],
    'Hyderabad':['mayank@blitznow.in','anil@blitznow.in','shankar.poddar@blitznow.in','chandrakanth.sunkoji@blitznow.in'],
    'Jaipur':['mayank@blitznow.in','sourabh.vijayvergia@blitznow.in','sourabh.vijay@blitznow.in','sanjay@blitznow.in']
}

city_dict2 = {
    'Bangalore': ['akshay@blitznow.in'],
    'Delhi':['akshay@blitznow.in'],
    'Mumbai':['akshay@blitznow.in'],
    'Hyderabad':['akshay@blitznow.in'],
    'Jaipur':['akshay@blitznow.in'],
}

In [12]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Your email credentials
sender_email = "akshay@blitznow.in"
sender_password = "Outofblitz1!"

# Function to send emails
def send_email(to_email_list, subject, body, attachment_paths):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ", ".join(to_email_list)
    msg['Subject'] = subject

    # Attach body
    msg.attach(MIMEText(body, 'html'))

    # Attach CSV files with updated names
    for attachment_path in attachment_paths:
        with open(attachment_path['path'], "rb") as file:
            part = MIMEApplication(file.read(), Name=f"{attachment_path['name']}_hub_report.csv")
            part['Content-Disposition'] = f'attachment; filename="{attachment_path["name"]}_hub_report.csv"'
            msg.attach(part)

    # Establish a connection to the SMTP server
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, to_email_list, msg.as_string())

# Iterate through city_dict
for city, email_list in city_dict2.items():
    # Initialize the HTML body for the email
    email_body = f"""
    <html>
      <head>
        <style>
          table {{
            border-collapse: collapse;
            width: 100%;
          }}
          th, td {{
            padding: 8px;
            text-align: center;
          }}
          th {{
            background-color: lightblue;
          }}
        </style>
      </head>
      <body>
        <h2>Hub-wise Report for {city}</h2>
    """

    # List to store paths of CSV files and their respective names
    csv_attachments = []

    # Iterate through each hub in the city
    for hub in df_report[df_report['City'] == city]['Hub'].unique():
        print(f"printing for {city} - {hub}")
        # Extract data for the current hub
        hub_data = df_report[(df_report['City'] == city) & (df_report['Hub'] == hub)].drop(columns=['City'])

        # Generate HTML table from DataFrame
        html_table = hub_data.to_html(index=False, escape=False)  # Added escape=False to prevent HTML escaping

        # Append the HTML table to the email body
        email_body += f"<h3>Hub: {hub}</h3>{html_table}"

        # Save DataFrame to CSV file
        csv_path = f"{city}_{hub}_hub_report.csv"
        hub_data.to_csv(csv_path, index=False)

        # Append CSV path and name to the list
        csv_attachments.append({'path': csv_path, 'name': f"{city}_{hub}"})

    # Close the HTML body
    email_body += """
      </body>
    </html>
    """

    # Send email to the city's email list with all CSV files attached
    send_email(email_list, f"Rider Performance Report for {city}", email_body, csv_attachments)


printing for Bangalore - BLDR-Franchise
printing for Bangalore - BLR FRH ECTY
printing for Bangalore - CMRJ-Franchise
printing for Bangalore - DS BLR BOMM
printing for Bangalore - DS BLR HBBL
printing for Bangalore - DS BLR MTH
printing for Bangalore - STNG-Franchise
printing for Bangalore - UUTR-Franchise
printing for Delhi - DEL FRH GKP
printing for Delhi - DS DEL DWK
printing for Delhi - DS DEL HKMN
printing for Delhi - DS DEL RNGR
printing for Delhi - GGKC-Franchise
printing for Delhi - ONFC-Franchise
printing for Delhi - RHNP-Franchise
printing for Mumbai - DS BOM KRL
printing for Mumbai - DS BOM SWRI
printing for Mumbai - DS BOM THAN
printing for Mumbai - GRGN-Franchise
printing for Hyderabad - DS HYD KMP
printing for Hyderabad - DS HYD SRNG
printing for Hyderabad - DS HYD TRNK
printing for Jaipur - DS JPR NPRG


2024-02-01 20:58:08,504| ERROR   | Socket exception: Operation timed out (60)
2024-02-01 20:58:10,032| ERROR   | Socket exception: Operation timed out (60)
